# SPACEL workflow (1/3): Deconvolution by Spoint on mouse brain ST dataset

July 2023

Dataset: 75 ST slices of mouse brain ([here](https://zenodo.org/record/8167488))

In [2]:
# import os
# os.environ['PYTORCH_DEVICE'] = 'cpu'
# os.environ['PYTORCH_MPS_ENABLE'] = '0'
# import torch
# torch.device('cpu')  # Set default device to CPU



import pandas as pd
import scanpy as sc
import anndata
import os
from tqdm import tqdm 
import scanpy as sc
import numpy as np
import sys



In [ ]:
from logging import warning
import socket

hostname = socket.gethostname()

if 'gbw-l-m2066' in hostname:
    data_root = '/Users/u0096223/Baidu/Science/Data PVDH'
elif 'r23i27n01' in hostname:
    data_root = '/lustre1/project/stg_00071/xhan/seq_processed/'
else:
    warning(f"Hostname {hostname} not recognized. Terminating.")

print(f"data_root set to: {data_root}")

data_root set to: /Users/u0096223/Baidu/Science/Data PVDH


## Load spatial transcriptomics data

In [ ]:
import sys
# sys.path.append('/Users/u0096223/Documents/GitHub/PVLabTime/Merscope')

data_dir = os.path.join(data_root, 'NovaST_data','NovaST_P5_LP_CTB_202503','deep_result')
### spatial data to be mapped ###
prefix =50 # resolution of the spatial data
spatial_file =os.path.join(data_dir,f'adata_thal_all_{prefix}.h5ad')

# anndata_filename = os.path.join(data_dir,'15-11-24_23-22_P5_mouse_brain_CBD_panel_Cells_annotated.h5ad')
# anndata_filename = os.path.join(data_dir,'Spoint_result_mouse_brain_st.h5ad')

### case 1: RetroSeq_xu data, P5 thalamocortical 
sc_files =os.path.join(data_root,'RNAseq_analysis','P5_thalamocortical_240715','P5_thalamocortical_Xcount.h5ad') # retroSeq xu
fig_root = os.path.join(data_dir,f'Figures_spatial_mapping_retroSeq_xu_{prefix}')

### case 2: P7 thalamus data, Giudice 2024
sc_files =os.path.join(data_root,'Public_data_transcriptomic','Lo_Giudice_2024_thalamus','PostNat_P7_control_raw_count_annotated.h5ad') # retroSeq xu
fig_root = os.path.join(data_dir,f'Figures_spatial_mapping_P7_thalamus_{prefix}')

# ### case 3: P5 Allen V1 data
# sc_files =os.path.join(data_root,'Public data transcriptomic','V1_P5_Allen_snRNA','Reference_V1_P5_CTX_Allen_snRNA_processed.h5ad') # retroSeq xu
# fig_root = os.path.join(data_dir,f'Figures_spatial_mapping_P5_Allen_V1_{prefix}')

if not os.path.exists(fig_root):
    os.makedirs(fig_root)



pn_trained_model= os.path.join(fig_root,'SPOINT_trained_model.h5')


The input data are anndata objects stored raw counts for scRNA-seq and ST. The scRNA-seq anndata must have cell type annotation in ``.obs``.

In [ ]:
adata = sc.read(spatial_file)
scadata = sc.read_h5ad(sc_files)

: 

In [4]:
scadata.obs

,projection,Sample,Age,Project,Condition,Timepoint,Nuclei,Combination,cell_id,class_label,...,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,n_genes,leiden
AAACCCAAGTCCCAAT.1_3,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LG,P7_Control_LG_Neurons,AAACCCAAGTCCCAAT.1_3,CS20230722_CLAS_18,...,0.693147,0.007857,25,3.258097,0.196433,1,0.693147,0.007857,3995,8
AAACCCAGTCCGCAGT.1_3,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LG,P7_Control_LG_Neurons,AAACCCAGTCCGCAGT.1_3,CS20230722_CLAS_18,...,0.693147,0.012650,23,3.178054,0.290955,1,0.693147,0.012650,3061,8
AAACGAAAGAGGCGTT.1_3,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LG,P7_Control_LG_Neurons,AAACGAAAGAGGCGTT.1_3,CS20230722_CLAS_18,...,0.000000,0.000000,29,3.401197,0.361731,1,0.693147,0.012473,3220,2
AAACGAAGTAGATGTA.1_3,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LG,P7_Control_LG_Neurons,AAACGAAGTAGATGTA.1_3,CS20230722_CLAS_18,...,0.693147,0.025151,16,2.833213,0.402414,0,0.000000,0.000000,2040,2
AAACGAAGTTAAGGAT.1_3,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LG,P7_Control_LG_Neurons,AAACGAAGTTAAGGAT.1_3,CS20230722_CLAS_18,...,0.000000,0.000000,16,2.833213,0.245173,0,0.000000,0.000000,2785,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTGTGACATCT.1_4,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LP,P7_Control_LP_Neurons,TTTGGTTGTGACATCT.1_4,CS20230722_CLAS_18,...,0.000000,0.000000,33,3.526361,0.449285,1,0.693147,0.013615,3175,14
TTTGTTGAGGAACGCT.1_4,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LP,P7_Control_LP_Neurons,TTTGTTGAGGAACGCT.1_4,CS20230722_CLAS_18,...,1.098612,0.025733,16,2.833213,0.205867,2,1.098612,0.025733,3203,4
TTTGTTGGTGGAATGC.1_4,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LP,P7_Control_LP_Neurons,TTTGTTGGTGGAATGC.1_4,CS20230722_CLAS_18,...,0.000000,0.000000,29,3.401197,0.300892,0,0.000000,0.000000,3652,9
TTTGTTGTCAAGGAGC.1_4,postnat_thal,postnat_thal,Postnat,DJ,Control,P7,LP,P7_Control_LP_Neurons,TTTGTTGTCAAGGAGC.1_4,CS20230722_CLAS_18,...,0.693147,0.012435,22,3.135494,0.273564,0,0.000000,0.000000,3200,12


In [5]:
scadata.var_names_make_unique()
scadata.obs_names_make_unique()

In [10]:
adata.X = adata.layers['raw']

In [17]:
# Replace adata.var_names with the actual gene names from adata.var['Gene']
adata.var_names = adata.var['Gene']
adata.var_names_make_unique()

## Initialize and train the Spoint model

In this step, we initialize the Spoint model using anndata objects for scRNA-seq and ST as input. The``celltype_key`` parameter represents the column name of the cell type annotation in the .obs attribute of the scRNA-seq anndata object. The ``sm_size`` parameter controls the number of simulated spots, and it is important to have a sufficient ``sm_size`` for accurate prediction. However, it should be noted that increasing the ``sm_size`` will also increase the simulation and training time. In general, we recommend setting ``sm_size`` to a value greater than 100,000.

In [11]:
import SPACEL
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint

Setting environment seed: 42


Global seed set to 0


In [ ]:
# Check that celltype_key exists and is valid
celltype_key = 'leiden'
if celltype_key not in scadata.obs.columns:
	raise ValueError(f"celltype_key '{celltype_key}' not found in scadata.obs columns: {scadata.obs.columns.tolist()}")

unique_celltypes = scadata.obs[celltype_key].unique()
if len(unique_celltypes) <= 1:
	raise ValueError(f"celltype_key '{celltype_key}' has {len(unique_celltypes)} unique values, expected > 1.")

# Check gene intersection
common_genes = set(scadata.var_names).intersection(set(adata.var_names))
if len(common_genes) == 0:
	raise ValueError("No common genes between scadata and adata.")

# Check sm_size
sm_size = 500000
if sm_size == 0:
	raise ValueError("sm_size must be greater than zero.")

spoint_model = Spoint.init_model(scadata, adata, celltype_key=celltype_key, sm_size=sm_size, use_gpu=False) #30min
# spoint_model = Spoint.init_model(scadata,adata,celltype_key='cluster_label',sm_size=500000,use_gpu=False) #30min

spoint_model.train(max_steps=5000, batch_size=1024) # 11h!!!

Setting global seed: 42
### Finding marker genes...
leiden
0     200
16    200
8     200
7     200
4     200
2     200
1     200
9     200
15    200
14    200
12    200
11    200
10    200
3     158
6     153
13    152
5     144
17    136
Name: count, dtype: int64
### Used gene numbers: 1454
### Initializing sample probability
### Genetating simulated spatial data using scRNA data with mode: unbalance
### Genetating simulated spatial data using scRNA data with mode: sqrt
### Genetating simulated spatial data using scRNA data with mode: balance


Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0


In [17]:
# save the model
import torch
torch.save(spoint_model.model.state_dict(), pn_trained_model)

In [ ]:
# to be tested, load spoint model
spoint_model.model.load_state_dict(torch.load(pn_trained_model))

Then, we utilize the trained model to predict the cell type composition of each spot in the spatial transcriptomics data. This prediction will generate a DataFrame object, where each row corresponds to a spot in the spatial transcriptomics data, each column represents a cell type from the single-cell RNA-seq data, and each entry indicates the proportion of a particular cell type in a spot. Additionally, we can obtain the anndata object of the spatial transcriptomics data with the deconvolution results embedded in the .obs attribute.

In [13]:
# run prediction 
pre = spoint_model.deconv_spatial()
st_ad = spoint_model.st_ad

In [14]:
st_ad.write(os.path.join(fig_root,'Spoint_result_mouse_brain_st.h5ad'))

In [15]:
import pandas as pd

# Save to CSV
pre.to_csv(os.path.join(fig_root,'spoint_model_deconv_spatial.csv'), index=False)